In [20]:
import cv2
import os
import mediapipe as mp
import time
import numpy as np
import pyvirtualcam
from pyvirtualcam import PixelFormat

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import preprocessing

import pickle

In [21]:
#Not Working
# Client conflict Manager
# Consult
# Contact needs more work
# Deaf
# Generation
# Grown
# Hard conflict recieve
# Huge
# Introduce
# Option doesnt work at certain angles
# Recieve maybe a little more
# to and unable conflict with huge and increase
# show conflicts option
# service conflicts introduce

In [22]:
model = pickle.load(open('models/trained/two_84', 'rb'))
device = torch.device("cpu")

model.to(device)

Sequential(
  (0): Linear(in_features=120, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): Linear(in_features=256, out_features=128, bias=True)
  (9): ReLU()
  (10): Linear(in_features=128, out_features=84, bias=True)
)

In [23]:
def collectData(handcords, base, lm, id,):
    if id == 0:
        base = [lm.x*resolution[1], lm.y*resolution[0]]
        return base
    else:
        cord = [lm.x*resolution[0]-base[0],lm.y*resolution[1]-base[1], lm.z * 100]
        handcords += cord
    

In [24]:
mpHands = mp.solutions.hands

hands = mpHands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5)

mpDraw = mp.solutions.drawing_utils

In [25]:
groups = os.listdir('Resources/')[1:]
classes = {}
for index, value in enumerate(groups):
    classes[index] = value

classes

{0: 'a',
 1: 'access',
 2: 'accustom',
 3: 'am',
 4: 'and',
 5: 'available',
 6: 'avenue',
 7: 'barrier',
 8: 'beautiful',
 9: 'break',
 10: 'by',
 11: 'bye',
 12: 'call',
 13: 'center',
 14: 'chat',
 15: 'client',
 16: 'commit',
 17: 'communicate',
 18: 'consult',
 19: 'contact',
 20: 'deaf',
 21: 'drop',
 22: 'everyone',
 23: 'family',
 24: 'finance',
 25: 'for',
 26: 'generation',
 27: 'give',
 28: 'goal',
 29: 'grown',
 30: 'hand',
 31: 'hard',
 32: 'have',
 33: 'hear',
 34: 'hello',
 35: 'help',
 36: 'hope',
 37: 'i',
 38: 'ILU',
 39: 'in',
 40: 'increase',
 41: 'introduce',
 42: 'live',
 43: 'manager',
 44: 'name',
 45: 'national',
 46: 'new',
 47: 'no',
 48: 'now',
 49: 'only',
 50: 'option',
 51: 'our',
 52: 'participate',
 53: 'people',
 54: 'please',
 55: 'program',
 56: 'project',
 57: 'quick',
 58: 'receive',
 59: 'result',
 60: 'retire',
 61: 'seek',
 62: 'show',
 63: 'success',
 64: 'support',
 65: 'team',
 66: 'thank',
 67: 'that',
 68: 'this',
 69: 'tiaa',
 70: 'time',


In [27]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)


unknownIndex = groups.index('unknown')
classification = unknownIndex
answer = torch.zeros(len(classes))
base = False

pTime = 0
cTime = 0
count = 0
sentence = ""

with pyvirtualcam.Camera(1280, 820, 20, fmt=PixelFormat.BGR) as cam:
    while True:

        success, img = cap.read()
        resolution = img.shape
        
        img = cv2.copyMakeBorder(img,100,0,0,0,cv2.BORDER_CONSTANT,value=[255,255,255])
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        handResults = hands.process(imgRGB)

        if handResults.multi_hand_landmarks:
            count += 1
            handcords = []

            for handLms in handResults.multi_hand_landmarks:
                for id, lm in enumerate(handLms.landmark):
                    if id == 0:
                        base = [lm.x*resolution[1], lm.y*resolution[0]]
                    else:
                        cord = [lm.x*resolution[1]-base[0],lm.y*resolution[0]-base[1], lm.z]

                        handcords += cord

                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

            handcords = np.array(handcords)
            for i in range(3):
                handcords[i::3] = preprocessing.normalize([handcords[i::3]])

            if len(handcords) != 120:
                handcords = np.concatenate((handcords, np.random.rand(60)))

            recordedCords = torch.Tensor(handcords)


            if count%10 == 0:
                # print(answer)
                
                try:
                    softAnswer = torch.nn.functional.softmax(answer, dim=1)
                    confIndex = torch.argmax(softAnswer)
                    if confIndex > len(classes): confIndex = unknownIndex
                    confidence = softAnswer[0][confIndex].item()
                    print(classes[confIndex.item()], confidence)
                except:
                    confidence = 0
                
                if confidence > 0.70:
                    answer = torch.argmax(answer)
                    classification = answer.item()
                    answer = torch.zeros(len(classes))
                else:
                    classification = unknownIndex
                    answer = torch.zeros(len(classes))

            else:
                # print(recordedCords)
                model_output = model(recordedCords.view(-1,120).to(device))
                
                softAnswer = torch.nn.functional.softmax(model_output, dim=1)
                confIndex = torch.argmax(softAnswer)
                if confIndex > len(classes): confIndex = unknownIndex
                confidence = softAnswer[0][confIndex].item()
                # print("hello world")
                
                if confidence > 0.015:
                    
                    answer = torch.Tensor.add(answer, model_output.to('cpu') )
                    if answer.shape[0] == 2:
                        answer = torch.Tensor.add(answer[0], answer[1])
                else:
                    if type(confIndex) == int:
                        print(classes[confIndex], confidence, "ads")
                    else:
                        print(classes[confIndex.item()], confidence, "ads")
                    
                    
                    unknownEye = torch.eye(len(classes))[unknownIndex] * .02
                    answer = torch.Tensor.add(answer, unknownEye)
                    if answer.shape[0] == 2:
                        answer = torch.Tensor.add(answer[0], answer[1])

            if classes[classification] != sentence.split(" ")[-1] and classes[classification] != 'unknown':
                sentence += " " + classes[classification]

            elif classes[classification] == "a" or len(sentence) > 45:
                sentence = ""
        # sentence = "hello this our team and our project"
        
        cv2.putText(img, sentence, (0,80), cv2.FONT_HERSHEY_PLAIN, 3, (0,0,0), 4)

        cTime= time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)) + f" {count}", (10,150), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255), 3)
        # img = cv2.flip(img, 1)
        cam.send(img)
        cam.sleep_until_next_frame()
        # cv2.imshow("img", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release() 

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
#[yes, other, no, fuck]
# {0:'bye', 1:'deaf', 2:'hello', 3:'home', 4:'I', 5:'no',  6:'unknown',  7:'yes',  8:'you'}

In [17]:
cap.release()